In [1]:
import os
import torch
from sklearn.model_selection import train_test_split

# Imports de la libreria propia
from vecopsciml.kernels.derivative import DerivativeKernels
from vecopsciml.utils import TensOps

# Imports de las funciones creadas para este programa
from models.constant_diffusivity import ConstantDiffusivityNeuralNetwork
from utils.folders import create_folder
from utils.load_data import load_data
from trainers.train import train_loop

In [2]:
# Creamos los paths para las distintas carpetas
ROOT_PATH = r'C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning'
DATA_PATH = os.path.join(ROOT_PATH, r'data\linear_homogeneous\linear_homogeneous.pkl')
RESULTS_FOLDER_PATH = os.path.join(ROOT_PATH, r'results\linear_homogeneous')
MODEL_RESULTS_PATH = os.path.join(ROOT_PATH, r'results\linear_homogeneous\modelo_prueba')

# Creamos las carpetas que sean necesarias (si ya están creadas se avisará de ello)
create_folder(RESULTS_FOLDER_PATH)
create_folder(MODEL_RESULTS_PATH)

Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\linear_homogeneous
Folder already exists at: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\results\linear_homogeneous\modelo_prueba


In [3]:
# Load dataset
dataset = load_data(DATA_PATH)

Data successfully loaded from: C:\Users\usuario\Desktop\rmunozTMELab\Physically-Guided-Machine-Learning\data\linear_homogeneous\linear_homogeneous.pkl


In [4]:
# Convolutional filters to derivate
dx = dataset['x_step_size']
dy = dataset['y_step_size']
D = DerivativeKernels(dx, dy, 0).grad_kernels_two_dimensions()

In [5]:
# Tratamiento de los datos para dividirlos en train y test
X_train = torch.Tensor(dataset['X_train']).unsqueeze(1)
y_train = TensOps(torch.tensor(dataset['y_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_train = TensOps(torch.tensor(dataset['k_train'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_val = torch.Tensor(dataset['X_val']).unsqueeze(1)
y_val = TensOps(torch.tensor(dataset['y_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)
K_val = TensOps(torch.tensor(dataset['k_val'], dtype=torch.float32, requires_grad=True).unsqueeze(1), space_dimension=2, contravariance=0, covariance=0)

X_np = X_train
y_np = y_train.values
K_np = K_train.values

X_train_np, X_test_np, y_train_np, y_test_np, K_train_np, K_test_np = train_test_split(X_np, y_np, K_np, test_size=0.2, random_state=42)

X_train = X_train_np
X_test = X_test_np

y_train = TensOps(y_train_np, space_dimension=y_train.space_dim, contravariance=0, covariance=0)
y_test = TensOps(y_test_np, space_dimension=y_train.space_dim, contravariance=0, covariance=0)

K_train = TensOps(K_train_np, space_dimension=K_train.space_dim, contravariance=0, covariance=0)
K_test = TensOps(K_test_np, space_dimension=K_train.space_dim, contravariance=0, covariance=0)

In [6]:
# Arquitectura del modelo
input_shape = X_train[0].shape  # [1, 10, 8]
hidden1_dim = 10
hidden2_dim = 10
output_shape = y_train.values[0].shape  # [1, 10, 10]

In [7]:
# Se carga el modelo y el optimizador
model = ConstantDiffusivityNeuralNetwork(input_shape, hidden1_dim, hidden2_dim, output_shape)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

In [8]:
# Parametros de entrenamiento
start_epoch = 0
n_epochs = 100

batch_size = 64
n_checkpoints = 10

In [9]:
train_loop(model, optimizer, n_checkpoints,
           X_train, y_train, X_test, y_test, D=D, 
           start_epoch=start_epoch, n_epochs=n_epochs, batch_size=batch_size, 
           model_results_path=MODEL_RESULTS_PATH
        )

Start training
Epoch 0, Train loss: 5.323e+09, Test loss: 1.630e+10, MSE(e): 3.113e+02, MSE(pi1): 2.086e+05, MSE(pi2): 1.509e+02, MSE(pi3): 1.235e+03
Epoch 10, Train loss: 5.200e+09, Test loss: 1.594e+10, MSE(e): 3.107e+02, MSE(pi1): 1.971e+05, MSE(pi2): 1.507e+02, MSE(pi3): 1.221e+03
Epoch 20, Train loss: 5.084e+09, Test loss: 1.560e+10, MSE(e): 3.100e+02, MSE(pi1): 1.862e+05, MSE(pi2): 1.505e+02, MSE(pi3): 1.209e+03
Epoch 30, Train loss: 4.973e+09, Test loss: 1.528e+10, MSE(e): 3.094e+02, MSE(pi1): 1.759e+05, MSE(pi2): 1.502e+02, MSE(pi3): 1.197e+03
Epoch 40, Train loss: 4.869e+09, Test loss: 1.497e+10, MSE(e): 3.088e+02, MSE(pi1): 1.662e+05, MSE(pi2): 1.500e+02, MSE(pi3): 1.185e+03
Epoch 50, Train loss: 4.770e+09, Test loss: 1.469e+10, MSE(e): 3.083e+02, MSE(pi1): 1.569e+05, MSE(pi2): 1.498e+02, MSE(pi3): 1.174e+03
Epoch 60, Train loss: 4.676e+09, Test loss: 1.441e+10, MSE(e): 3.077e+02, MSE(pi1): 1.482e+05, MSE(pi2): 1.496e+02, MSE(pi3): 1.164e+03
Epoch 70, Train loss: 4.587e+09, T